In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import re
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import FeatureUnion
from sklearn.model_selection import train_test_split

C:\Users\Cxovrika\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
df_legitimate = pd.read_csv("clean_legitimate.txt",names=['url'], header = None, sep = 'asdas2131sad12sa1a11a2sa1',engine='python',encoding = 'utf8').dropna()
df_malicious = pd.read_csv("clean_malicious.txt",names=['url'], header = None, sep = 'asdas2131sad12sa1a11a2sa1',engine='python',encoding = 'utf8').dropna()
X_data = df_legitimate['url'].tolist() + df_malicious['url'].tolist()
y_data = np.zeros(len(df_legitimate)).tolist() + np.ones(len(df_malicious)).tolist()

In [3]:
print(df_malicious.shape)
print(df_legitimate.shape)

(44698, 1)
(1265994, 1)


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=0, shuffle = True)

In [5]:
# marto sityvebis da ricxvebis datoveba
X_train_words = [re.sub("[^A-Za-z0-9]", " ", i) for i in X_train]
X_train_words = [re.sub(' +',' ', i).strip().lower() for i in X_train_words]  
X_test_words = [re.sub("[^A-Za-z0-9]", " ", i) for i in X_test]
X_test_words = [re.sub(' +',' ', i).strip().lower() for i in X_test_words] 

In [6]:
cv = CountVectorizer()
tf_idf = TfidfVectorizer()

x_traincv =cv.fit_transform(X_train_words)
x_testcv=cv.transform(X_test_words)

x_train_tf_idf =tf_idf.fit_transform(X_train_words)
x_test_tf_idf=tf_idf.transform(X_test_words)

y_train= [int(i) for i in y_train]
y_test= [int(i) for i in y_test]

In [7]:
#abrunebs featurebis raodenoba X data Size zomis np.arrays
def get_additional_features(data):
    
    a1  = np.asarray([[int(len(i)<=4) for i in data]])
    a2  = np.asarray([[int(4<len(i)<=12) for i in data]])
    a3  = np.asarray([[int(12<len(i)<=44) for i in data]])
    a4  = np.asarray([[int(44<len(i)) for i in data]])

    a5  = np.asarray([[i.count('/') for i in data]])
    a6  = np.asarray([[i.count('//') for i in data]])
    a7  = np.asarray([[i.count('<') for i in data]])
    a8  = np.asarray([[i.count('>') for i in data]])
    a9  = np.asarray([[i.count('.') for i in data]])
    a10 = np.asarray([[i.count(',') for i in data]])
    a11 = np.asarray([[i.count('..') for i in data]])
    a12 = np.asarray([[i.count('(') for i in data]])
    a13 = np.asarray([[i.count(')') for i in data]])
    a14 = np.asarray([[i.count('[') for i in data]])
    a15 = np.asarray([[i.count(']') for i in data]])
    a16 = np.asarray([[i.count('"') for i in data]])
    a17 = np.asarray([[i.count("'") for i in data]])
    a18 = np.asarray([[i.count('?') for i in data]])
    a19 = np.asarray([[i.count(';') for i in data]])
    a20 = np.asarray([[i.count('-') for i in data]])
    
    a21 = np.asarray([[i.count('https') for i in data]])
    a22 = np.asarray([[(i.count('<')>0 and i.count('>')) for i in data]])
    a23 = np.asarray([[i.count('%') for i in data]])
    a24 = np.asarray([[i.count('+') for i in data]])
    a25 = np.asarray([[i.count('</') for i in data]])
    a26 = np.asarray([[i.count('.exe') for i in data]])
    a27 = np.asarray([[i.count('&') for i in data]])
    a28 = np.asarray([[i.count('=') for i in data]])

    
    return np.concatenate([a1, a2, a3, a4, a5, a6, a7, a8, a9, a10, a11, a12, a13, a14, a15, a16, a17, a18, a19, a20, a21, a22, a23, a24,a25,a26,a27,a28]).T

In [8]:
# featurebis gaertianeba
X_train_final = sp.sparse.hstack([x_traincv, get_additional_features(X_train)])
X_test_final = sp.sparse.hstack([x_testcv, get_additional_features(X_test)])

In [9]:

mnb_model = MultinomialNB()
linear_svc_model = LinearSVC()
logistic_model = LogisticRegression()
random_forest_model = RandomForestClassifier(verbose=3)
svc_model = SVC(verbose=2)

In [10]:
mnb_model.fit(X_train_final, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [13]:
linear_svc_model.fit(X_train_final, y_train) #~2min

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [14]:
logistic_model.fit(X_train_final, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [11]:
random_forest_model.fit(X_train_final, y_train) # ~30min

building tree 1 of 10


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.9min remaining:    0.0s


building tree 2 of 10


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  5.8min remaining:    0.0s


building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed: 30.8min finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=3,
            warm_start=False)

In [ ]:
svc_model.fit(X_train_final, y_train)#too long

[LibSVM]

In [13]:
def confusion(model, X,y):
    predictions=model.predict(X)

    tn, fp, fn, tp = confusion_matrix(predictions, y).ravel()
    total = tn+fp+fn+tp
    actual_no = tn + fp
    actual_yes = tp + fn

    print('tn:' + str(tn), 'fp:' + str(fp))
    print('fn:' + str(fn), 'tp:' + str(tp) +'\n')
    print('Accuracy: ' + str((tp+tn)*100/total))
    print('Misclassification: ' + str((fp+fn)*100/total))
    print('True Positive Rate: ' + str((tp)*100/actual_yes))
    print('False Positive Rat: ' + str((fp)*100/actual_no))
    print('Specificity: ' + str((tn)*100/actual_no))
    print('Precision: ' + str((tp)*100/(tp+fp)))
    print('Prevalence: ' + str(actual_yes*100/total))
    print(tp / (tp+fp))
    print(tn / (tn+fn))

In [17]:
confusion(mnb_model, X_test_final, y_test)

tn:252576 fp:1589
fn:630 tp:7344

Accuracy: 99.15350253109992
Misclassification: 0.8464974689000874
True Positive Rate: 92.09932279909707
False Positive Rat: 0.6251844274388685
Specificity: 99.37481557256113
Precision: 82.21202283667301
Prevalence: 3.0418976192020266
0.8221202283667302
0.9975119073007749


In [18]:
confusion(linear_svc_model, X_test_final, y_test)

tn:253170 fp:171
fn:36 tp:8762

Accuracy: 99.9210342604496
Misclassification: 0.0789657395503912
True Positive Rate: 99.59081609456695
False Positive Rat: 0.0674979572986607
Specificity: 99.93250204270134
Precision: 98.08574946826374
Prevalence: 3.3562346693929554
0.9808574946826374
0.99985782327433


In [19]:
confusion(logistic_model, X_test_final, y_test)

tn:253161 fp:266
fn:45 tp:8667

Accuracy: 99.88136065217309
Misclassification: 0.11863934782691625
True Positive Rate: 99.48347107438016
False Positive Rat: 0.10496119198033359
Specificity: 99.89503880801966
Precision: 97.02227695063249
Prevalence: 3.3234276471642907
0.9702227695063249
0.9998222790929124


In [ ]:
confusion(svc_model, X_test_final, y_test)

In [14]:
confusion(random_forest_model, X_test_final, y_test)

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    4.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   20.2s finished


tn:253175 fp:306
fn:31 tp:8627

Accuracy: 99.87144225010395
Misclassification: 0.1285577498960475
True Positive Rate: 99.64194964194964
False Positive Rat: 0.12071910715201534
Specificity: 99.87928089284799
Precision: 96.57449904847196
Prevalence: 3.3028278890207106
0.9657449904847196
0.9998775700417841
